In [24]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
import lightning as L
import yaml
import lightning_scripts.lightning_classifier as lightning 
import importlib
import torch


KeyboardInterrupt



In [25]:
torch.set_float32_matmul_precision = 'medium'

In [26]:
importlib.reload(lightning)
LitWordAudioSetModel = lightning.LitWordAudioSetModel

In [27]:
###
config_path = "lightning_scripts/configs/word_audioset_resnet50.yaml"
config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)
config['num_workers'] = 2
config['hparas']['batch_size'] = 256

In [28]:
config

{'data': {'root': '/mnt/ceph/users/jfeather/data/training_datasets_audio/JSIN_all_v3/subsets/',
  'target_keys': ['signal/word_int', 'noise/labels_binary_via_int']},
 'audio_rep': {'name': 'cochleagram_1', 'on_gpu': True},
 'audio_transforms': {'low_snr': -10, 'high_snr': 10, 'dbspl': 60},
 'val_metric': {'word_task': 'signal/word_int',
  'audioset_task': 'noise/labels_binary_via_int'},
 'hparas': {'epochs': 6,
  'batch_size': 256,
  'step_lr': 2,
  'optimizer': 'AdamW',
  'lr': 0.1,
  'valid_step': 5000,
  'task_loss_params': {'signal/word_int': {'loss_type': 'crossentropyloss',
    'weight': 1.0},
   'noise/labels_binary_via_int': {'loss_type': 'bcewithlogitsloss',
    'weight': 300.0}}},
 'model': {'arch_name': 'resnet_multi_task50',
  'arch_params': {'num_classes': {'signal/word_int': 794,
    'noise/labels_binary_via_int': 517},
   'pretrained': False}},
 'num_workers': 2}

In [29]:
model = LitWordAudioSetModel(config)

In [30]:
# model

In [31]:
trainer = L.Trainer(
    # limit_train_batches=5,
    # limit_val_batches=2,
    max_epochs=config['hparas']['epochs'],
    strategy='ddp_notebook',
    devices=2)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [32]:
trainer.fit(model)

You are using a CUDA device ('NVIDIA H100 PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
You are using a CUDA device ('NVIDIA H100 PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

torch.Size([256, 1, 40000])
torch.Size([256, 1, 40000])torch.Size([256, 1, 40000])
torch.Size([256, 1, 40000])

torch.Size([256, 1, 40000])
torch.Size([256, 1, 40000])
torch.Size([256, 1, 40000])
torch.Size([256, 1, 40000])
torch.Size([256, 1, 40000])
torch.Size([256, 1, 40000])


/mnt/home/igriffith/ceph/conda_envs/cochdnn_ssl_pl/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_signal/word_int_acc', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
/mnt/home/igriffith/ceph/conda_envs/cochdnn_ssl_pl/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_noise/labels_binary_via_int_acc', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.


torch.Size([256, 1, 40000])
torch.Size([256, 1, 40000])


/mnt/home/igriffith/ceph/conda_envs/cochdnn_ssl_pl/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (45) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

W0917 19:11:28.337000 23456247908160 torch/multiprocessing/spawn.py:146] Terminating process 590347 via signal SIGTERM
[rank: 0] Received SIGTERM: 15
W0917 19:11:58.967000 23456247908160 torch/multiprocessing/spawn.py:154] Unable to shutdown process 590347 via SIGTERM , forcefully exiting via SIGKILL


ProcessRaisedException: 

-- Process 1 terminated with the following error:
Traceback (most recent call last):
  File "/mnt/home/igriffith/ceph/conda_envs/cochdnn_ssl_pl/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1131, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/home/igriffith/ceph/conda_envs/cochdnn_ssl_pl/lib/python3.12/queue.py", line 180, in get
    self.not_empty.wait(remaining)
  File "/mnt/home/igriffith/ceph/conda_envs/cochdnn_ssl_pl/lib/python3.12/threading.py", line 359, in wait
    gotit = waiter.acquire(True, timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/home/igriffith/ceph/conda_envs/cochdnn_ssl_pl/lib/python3.12/site-packages/torch/utils/data/_utils/signal_handling.py", line 67, in handler
    _error_if_any_worker_fails()
RuntimeError: DataLoader worker (pid 590708) is killed by signal: Killed. 

The above exception was the direct cause of the foll